In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import csv


def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_dataset(dataset, look_back=1, time_steps=1):
    X, Y = [], []
    for i in range(len(dataset) - time_steps - look_back + 1):
        a = dataset[i:(i + time_steps * look_back), 0].reshape(time_steps, look_back)
        X.append(a)
        Y.append(dataset[i + time_steps * look_back - 1, 0])
    return np.array(X), np.array(Y)

def extract_lat_lon(filename):
    lat_str = filename.split("lat_")[1].split("_")[0]
    lon_str = filename.split("lon_")[1].split(".")[0]
    lat = float(lat_str)
    lon = float(lon_str)
    return lat, lon


folder_path = 'Replace the folder with the csv files'
date = 'This is the date string Ex: For Day10: d10'

# Following is the output csv file 
with open(f"Statistics for LSTM {date}.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f'Statistics for LSTM {date}'])
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        print(f'Processing file: {file_path}')
        tag = ''
        if 'dbiastg' in file_name:
            tag = 'dbiastg'
        else:
            tag = 'biastg'   
        
        writer.writerow([file_name])   
        latitude, longitude = extract_lat_lon(file_name)
        writer.writerow(['latitude',latitude])
        writer.writerow(['longitude',longitude])
        data = pd.read_csv(file_path, index_col='date', parse_dates=True)
        data = data[tag]
                
        if data.notnull().any():  # Proceed if there's at least one non-null value
            # Prepare the data
            data = data.asfreq('B').fillna(method='ffill')  # Fill missing values
            data = data.dropna()
            # Normalize the dataset
            scaler = MinMaxScaler(feature_range=(0, 1))
            data = scaler.fit_transform(data.values.reshape(-1, 1))

            # Split the data into training and testing sets
            train_size = int(len(data) * 0.8)
            test_size = len(data) - train_size
            train_data, test_data = data[0:train_size,:], data[train_size:len(data),:]

            # Reshape the data into X=t and Y=t+time_steps
            look_back = 1
            time_steps = 3  # Increase the number of time steps
            X_train, Y_train = create_dataset(train_data, look_back, time_steps)
            X_test, Y_test = create_dataset(test_data, look_back, time_steps)

            # Reshape the input to be [samples, time steps, features]
            X_train = np.reshape(X_train, (X_train.shape[0], time_steps, look_back))
            X_test = np.reshape(X_test, (X_test.shape[0], time_steps, look_back))

            # Create and fit the Bidirectional LSTM network with additional hidden layers
            model = Sequential()
            model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(time_steps, look_back))))
            model.add(Dropout(0.2))  # Add dropout layer
            model.add(Bidirectional(LSTM(32, return_sequences=True)))  # Add second Bidirectional LSTM layer
            model.add(Dropout(0.2))  # Add dropout layer
            model.add(Bidirectional(LSTM(16)))  # Add third Bidirectional LSTM layer
            model.add(Dropout(0.2))  # Add dropout layer
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

            # Make predictions
            train_predict = model.predict(X_train)
            test_predict = model.predict(X_test)

            # Invert predictions
            train_predict = scaler.inverse_transform(train_predict)
            Y_train = scaler.inverse_transform([Y_train])
            test_predict = scaler.inverse_transform(test_predict)
            Y_test = scaler.inverse_transform([Y_test])

            # Evaluate the model
            mae = mean_absolute_error(Y_test[0], test_predict[:,0])
            mse = mean_squared_error(Y_test[0], test_predict[:,0])
            rmse = np.sqrt(mse)
            mbe = np.mean(Y_test[0] - test_predict[:,0])
            mape = mean_absolute_percentage_error(Y_test[0], test_predict[:,0])

            print(f"RMSE: {rmse:.2f}")
            writer.writerow(['RMSE',rmse])
            print(f"MAE: {mae:.2f}")
            writer.writerow(['MAE',mae])
            print(f"MBE: {mbe:.2f}")
            writer.writerow(['MBE',mbe])
            print(f"MAPE: {mape:.2f}%")
            writer.writerow(['MAPE',mape])

            # Plot the results
            plt.figure(figsize=(14, 6))
            plt.plot(data, label='Observed')
            plt.plot(np.concatenate((train_predict, test_predict)), color='r', label='Predicted')
        else:
            print(f'Skipping file {file_name} due to no non-null values in {tag} column.')
            continue



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
import os
import csv

# Function to calculate Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Function to prepare the dataset for LSTM
def create_dataset(dataset, look_back=1, time_steps=1):
    X, Y = [], []
    for i in range(len(dataset) - time_steps - look_back + 1):
        a = dataset[i:(i + time_steps * look_back), 0].reshape(time_steps, look_back)
        X.append(a)
        Y.append(dataset[i + time_steps * look_back - 1, 0])
    return np.array(X), np.array(Y)

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9  # Reduce learning rate after 10 epochs

# Function to build the LSTM model
def build_lstm_model(units=64, dropout_rate=0.2, time_steps=3, look_back=1):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(time_steps, look_back)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units // 2, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units // 4))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam())
    return model

folder_path = 'Replace the folder with the csv files'
date = 'This is the date string Ex: For Day10: d10'

# Hyperparameter search space
param_grid = {
    'units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.5],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100],
    'time_steps': [3, 5],  # Number of time steps
}

# Prepare CSV output for logging results
with open(f"Statistics for LSTM Hyperparameter Tuning {date}.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f'Statistics for LSTM Hyperparameter Tuning {date}'])
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        print(f'Processing file: {file_path}')
        tag = ''
        if 'dbiastg' in file_name:
            tag = 'dbiastg'
        else:
            tag = 'biastg'   
        
        writer.writerow([file_name])   
        data = pd.read_csv(file_path, index_col='time', parse_dates=True)
        data = data[tag]
                
        if data.notnull().any():  # Proceed if there's at least one non-null value
            # Prepare the data
            data = data.asfreq('B').fillna(method='ffill')  # Fill missing values
            data = data.dropna()
            # Normalize the dataset
            scaler = MinMaxScaler(feature_range=(0, 1))
            data = scaler.fit_transform(data.values.reshape(-1, 1))

            # Split the data into training and testing sets
            train_size = int(len(data) * 0.8)
            test_size = len(data) - train_size
            train_data, test_data = data[0:train_size,:], data[train_size:len(data),:]

            # Reshape the data into X=t and Y=t+time_steps
            look_back = 1
            time_steps = 3  # Increase the number of time steps
            X_train, Y_train = create_dataset(train_data, look_back, time_steps)
            X_test, Y_test = create_dataset(test_data, look_back, time_steps)

            # Reshape the input to be [samples, time steps, features]
            X_train = np.reshape(X_train, (X_train.shape[0], time_steps, look_back))
            X_test = np.reshape(X_test, (X_test.shape[0], time_steps, look_back))

            # Hyperparameter tuning using RandomizedSearchCV
            model = KerasRegressor(build_fn=build_lstm_model, verbose=0)
            random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=5, cv=3, verbose=2, random_state=42)
            random_search.fit(X_train, Y_train)

            best_params = random_search.best_params_
            print(f"Best hyperparameters: {best_params}")

            # Create the final model with best hyperparameters
            best_model = build_lstm_model(units=best_params['units'], dropout_rate=best_params['dropout_rate'], time_steps=time_steps, look_back=look_back)
            
            # Implement EarlyStopping and Learning Rate Scheduler
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            lr_scheduler = LearningRateScheduler(scheduler)

            # Train the model
            best_model.fit(X_train, Y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2, validation_data=(X_test, Y_test), callbacks=[early_stopping, lr_scheduler])

            # Make predictions
            train_predict = best_model.predict(X_train)
            test_predict = best_model.predict(X_test)

            # Invert predictions
            train_predict = scaler.inverse_transform(train_predict)
            Y_train = scaler.inverse_transform([Y_train])
            test_predict = scaler.inverse_transform(test_predict)
            Y_test = scaler.inverse_transform([Y_test])

            # Evaluate the model
            mae = mean_absolute_error(Y_test[0], test_predict[:,0])
            mse = mean_squared_error(Y_test[0], test_predict[:,0])
            rmse = np.sqrt(mse)
            mbe = np.mean(Y_test[0] - test_predict[:,0])
            mape = mean_absolute_percentage_error(Y_test[0], test_predict[:,0])

            print(f"RMSE: {rmse:.2f}")
            writer.writerow(['RMSE', rmse])
            print(f"MAE: {mae:.2f}")
            writer.writerow(['MAE', mae])
            print(f"MBE: {mbe:.2f}")
            writer.writerow(['MBE', mbe])
            print(f"MAPE: {mape:.2f}%")
            writer.writerow(['MAPE', mape])

            # Plot the results
            plt.figure(figsize=(14, 6))
            plt.plot(data, label='Observed')
            plt.plot(np.concatenate((train_predict, test_predict)), color='r', label='Predicted')
            plt.title(f'Observed vs Predicted for {file_name}')
            plt.xlabel('Date')
            plt.ylabel(tag)
            plt.legend()
            plt.show()
        else:
            print(f'Skipping file {file_name} due to no non-null values in {tag} column.')
            continue
